In [16]:
import stim
import numpy as np
from itertools import product
import tomography_util as tm

# Single-qubit basis rotations before Z measurement for tomography
BASIS_ROTATIONS = {
    'I': [],
    'X': ['H'],
    'Y': ['S_DAG', 'H'],
    'Z': []
}

def apply_basis_rotation(circuit, qubit, basis):
    for gate in BASIS_ROTATIONS.get(basis, []):
        if gate == 'H':
            circuit.append("H", [qubit])
        elif gate == 'S_DAG':
            circuit.append("S_DAG", [qubit])
        else:
            raise ValueError(f"Unknown gate {gate}")

def pauli_twirled_params(t1, t2, t_gate=1.0):
    """
    Approximate Pauli channel probabilities from T1 and T2 times.å
    """
    p_damp = 1 - np.exp(-t_gate / t1)
    p_dephase = 1 - np.exp(-t_gate / t2)

    p_x = p_y = p_damp / 2
    p_z = max(0, p_dephase - p_damp / 2)
    return p_x, p_y, p_z


SHOTS = 1
BASE_ERROR = 0.1
POSTSELECTION = False
bases = ['X', 'Y', 'Z']

syndrome_rounds = 1
total_samples = {term:[] for term in bases}
gamma = .01
lam = .05

p_x = p_y = gamma / 2
p_z = max(0, lam - gamma / 2)
for basis in bases:
    circuit = stim.Circuit()
    # Prepare Bell state
    circuit.append("H", [0])
    circuit.append("CNOT", [0, 1])
    # Apply twirled Pauli noise
    circuit.append("PAULI_CHANNEL_1", [0], [p_x, p_y, p_z])
    circuit.append("PAULI_CHANNEL_1", [1], [p_x, p_y, p_z])
    # Basis rotations for tomography measurement
    apply_basis_rotation(circuit, 0, basis)
    apply_basis_rotation(circuit, 1, basis)
    circuit.append("M", [0, 1])
    sampler = circuit.compile_sampler()
    samples = sampler.sample(shots=SHOTS)
    total_samples[basis].append(samples)

        
measurements = {term:[] for term in bases}
for basis in bases:
    measurements[basis] = ([total_samples[basis].count(True), total_samples[basis].count(False)])
print(measurements)

rho = tm.state_tomography_mle(measurements)
print("Estimated density matrix:")
print(rho)

print("\nTrace:", np.trace(rho))
print("Hermitian:", np.allclose(rho, rho.conj().T))
print("Eigenvalues:", np.linalg.eigvalsh(rho))

tm.plot_density_matrix(rho)



AttributeError: 'numpy.ndarray' object has no attribute 'count'